## Off policy Q-learning and Sarsa algorithm


In [0]:
import numpy as np
np.set_printoptions(precision=3)

class Environment():
    
    def __init__(self, epsilon=0.9, episodes=100, max_steps=100, alpha=0.085, gamma=0.9):
        self.epsilon = epsilon      # e-greedy
        self.num_episodes = episodes
        self.max_steps = max_steps
        self.alpha = alpha
        self.gamma = gamma
        self.state = None
        self.reward = 0
        self.state_space = 5*4
        self.action_space = 4
        self.Q = np.zeros((self.state_space, self.action_space), np.float32)

    def next_action(self, state, exploitation=False):       
        if np.random.uniform(0,1) < self.epsilon and exploitation == False: 
            action = np.random.randint(self.action_space) # Explore
        else: 
            action = np.argmax(self.Q[state, :])          # Exploit
        return action 
  
    def next_state(self, state, action):
        x = int(state/5)
        y = state%5
    
        old_x,old_y = x,y
            
        # Set up Next-State
        if (action == 0):   # Up
            if (x != 0):
                x = x-1
        
        elif (action == 1): # Right
            if (y != 4):
                y = y+1
        
        elif (action == 2): # Down
            if (x != 3):
                x = x+1      
        
        elif (action == 3): # Left
            if (y != 0):
                y = y-1
                                        
        self.state = x*5+y
        
        # Set up Reward according to rules
        if ((x,y) == (old_x,old_y)): # Off-Grid
            reward = -1
        elif ((x,y) == (0,0)):       # Goal-State
            reward = 10
        elif ((x,y) == (1,3)):       # F-Position
            reward = -5
        else:
            reward = 0

        if ((old_x,old_y) == (1,3)):       # F-Position
            self.state_visualization[old_x][old_y] = "F"
        else:
            self.state_visualization[old_x][old_y] = "-"
        self.state_visualization[x][y] = "S"

        return self.state, reward

    def updateQ(self, current_state, next_state, reward, current_action, next_action):
        Q = self.Q[current_state][current_action] 
        Qnext = (reward + self.gamma * self.Q[next_state][next_action])
        self.Q[current_state][current_action] = (1-self.alpha)*Q + self.alpha*Qnext
        
    def reset(self):
        self.state_visualization = np.array([['G', '-' ,'-' ,'-', '-'],
                              ['-', '-' ,'-' ,'F', '-'],
                              ['-', '-' ,'-' ,'-', '-'],
                              ['-', '-' ,'-' ,'-', 'S']])
        # Initial State ('S')
        self.state = 19
        return self.state
        
    def is_goal_state(self):
        return self.state == 0
    
    def actions_map(self, i):
        if (i==0):
            return "Up"
        elif (i==1):
            return "Right"
        elif (i==2):
            return "Down"
        elif (i==3):
            return "Left"

### Q-Learning Algorithm (off-policy TD)

In [0]:
# Q Algorithm (off-policy TD)

# Generate the Environment
environment = Environment(episodes=300, max_steps=100)

for episode in range(environment.num_episodes): 
    t = 0
    
    # Initialize S
    current_state = environment.reset()

    # Repeat for each step of episode
    while t < environment.max_steps and not environment.is_goal_state(): 
        
        # Select 'a' using policy derived from Q (e-greedy)
        current_action = environment.next_action(current_state) 
        
        # Take action 'a', observe immediate reward and new state
        next_state, reward = environment.next_state(current_state, current_action)

         #print(environment.state_visualization, ",   ", 
         #      environment.actions_map(current_action), 
         #      ", reward:", reward, "\n\n")

        # Select next-action by using the exploitation flag to True so that we
        # get the maximum Qnext in the update-rule
        next_action = environment.next_action(next_state, exploitation=True)
        
        # Update Q
        environment.updateQ(current_state, next_state, reward, current_action, next_action)
  
        current_state = next_state
          
        t += 1
        
# print("\n\n", environment.Q)

### Play once in deterministic mode

In [0]:
# Play Once
exploitation = True
current_state = environment.reset()
current_action = environment.next_action(current_state, exploitation) 
  
# Repeat until agent reaches goal state or max_steps
while t < environment.max_steps and not environment.is_goal_state():

    next_state, reward = environment.next_state(current_state, current_action)

    print(environment.state_visualization, ",   ", 
          environment.actions_map(current_action), 
          ", reward:", reward, "\n\n")

    next_action = environment.next_action(next_state, exploitation)

    current_state = next_state
    current_action = next_action

    t += 1


[['G' '-' '-' '-' '-']
 ['-' '-' '-' 'F' '-']
 ['-' '-' '-' '-' 'S']
 ['-' '-' '-' '-' '-']] ,    Up , reward: 0 


[['G' '-' '-' '-' '-']
 ['-' '-' '-' 'F' 'S']
 ['-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-']] ,    Up , reward: 0 


[['G' '-' '-' '-' 'S']
 ['-' '-' '-' 'F' '-']
 ['-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-']] ,    Up , reward: 0 


[['G' '-' '-' 'S' '-']
 ['-' '-' '-' 'F' '-']
 ['-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-']] ,    Left , reward: 0 


[['G' '-' 'S' '-' '-']
 ['-' '-' '-' 'F' '-']
 ['-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-']] ,    Left , reward: 0 


[['G' 'S' '-' '-' '-']
 ['-' '-' '-' 'F' '-']
 ['-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-']] ,    Left , reward: 0 


[['S' '-' '-' '-' '-']
 ['-' '-' '-' 'F' '-']
 ['-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-']] ,    Left , reward: 10 




In [0]:
print("Rows: States [0,1,2,..,19]")
print("Columns: Up, Right, Down, Left")
print(environment.Q)


Rows: States [0,1,2,..,19] 
Columns: Up, Right, Down, Left

[[ 0.     0.     0.     0.   ]
 [ 7.999  8.1    8.099 10.   ]
 [ 7.1    7.29   7.29   9.   ]
 [ 6.29   6.561  1.56   8.1  ]
 [ 5.56   5.561  5.904  7.29 ]
 [ 9.998  8.058  7.7    7.818]
 [ 9.     7.29   7.285  8.991]
 [ 8.1    1.561  6.56   8.1  ]
 [ 7.29   5.902  5.902  7.29 ]
 [ 6.561  4.904  5.313  1.561]
 [ 8.986  7.242  7.003  6.894]
 [ 8.1    6.544  6.495  8.055]
 [ 7.29   5.903  5.892  7.287]
 [ 1.561  5.312  5.308  6.561]
 [ 5.905  4.312  4.781  5.905]
 [ 8.061  6.349  5.936  6.054]
 [ 7.27   5.843  5.441  7.117]
 [ 6.56   5.277  4.863  6.486]
 [ 5.904  4.778  4.306  5.896]
 [ 5.314  3.782  3.782  5.313]]


In [0]:
print("Values-States [4,5]")
print(np.reshape(np.max(environment.Q, axis=1), [4, 5]))

Values-States [4,5]
[[ 0.    10.     4.298  0.455 -2.083]
 [10.     4.016  2.019 -2.162 -2.995]
 [ 3.939  1.197  0.325 -0.744 -2.205]
 [ 1.348  0.055 -0.83  -0.836 -1.662]]


### SARSA Algorithm (on-policy TD)

In [0]:
# SARSA Algorithm (on-policy TD)

# Generate the Environment
environment = Environment(episodes=300, max_steps=100)

for episode in range(environment.num_episodes): 
    t = 0
    
    # Initialize S
    current_state = environment.reset()
    # Select 'a' using policy derived from Q (e-greedy)
    current_action = environment.next_action(current_state) 
  
    # Repeat for each step of episode
    while t < environment.max_steps and not environment.is_goal_state(): 
        
        # Take action 'a', observe immediate reward and new state
        next_state, reward = environment.next_state(current_state, current_action)

         #print(environment.state_visualization, ",   ", 
         #      environment.actions_map(current_action), 
         #      ", reward:", reward, "\n\n")

        # Select next action using policy derived from Q (e-greedy)
        next_action = environment.next_action(next_state)

        # Update Q
        environment.updateQ(current_state, next_state, reward, current_action, next_action)
  
        current_state = next_state
        current_action = next_action
          
        t += 1

### Play once in deterministic mode

In [0]:
# Play Once
exploitation = True
current_state = environment.reset()
current_action = environment.next_action(current_state, exploitation) 
  
# Repeat until agent reaches goal state or max_steps
while t < environment.max_steps and not environment.is_goal_state():

    next_state, reward = environment.next_state(current_state, current_action)

    print(environment.state_visualization, ",   ", 
          environment.actions_map(current_action), 
          ", reward:", reward, "\n\n")

    next_action = environment.next_action(next_state, exploitation)

    current_state = next_state
    current_action = next_action

    t += 1

In [0]:
print("\nRows: States [0,1,2,..,19] \nColumns: Up, Right, Down, Left\n")
print(environment.Q)


Rows: States [0,1,2,..,19] 
Columns: Up, Right, Down, Left

[[ 0.     0.     0.     0.   ]
 [ 2.788  0.246  2.089 10.   ]
 [-1.878 -2.618  0.078  4.298]
 [-3.683 -3.041 -7.041  0.455]
 [-3.929 -3.819 -3.6   -2.083]
 [10.     1.231  2.146  3.599]
 [ 2.775 -1.916  0.624  4.016]
 [ 0.396 -7.356 -0.91   2.019]
 [-2.998 -3.1   -2.162 -2.386]
 [-2.995 -4.715 -3.032 -7.296]
 [ 3.939  0.433 -0.222  1.189]
 [ 0.889 -0.642 -0.489  1.197]
 [-0.646 -3.077 -0.98   0.325]
 [-7.422 -2.719 -1.979 -0.744]
 [-4.12  -3.884 -2.205 -2.231]
 [ 1.348 -0.216 -1.033 -1.466]
 [ 0.055 -1.107 -1.702 -0.296]
 [-0.83  -1.915 -2.115 -0.856]
 [-3.44  -2.415 -3.046 -0.836]
 [-2.642 -3.307 -3.265 -1.662]]


In [0]:
print("Values-States [4,5]")
print(np.reshape(np.max(environment.Q, axis=1), [4, 5]))

Values-States [4,5]
[[ 0.    10.     4.298  0.455 -2.083]
 [10.     4.016  2.019 -2.162 -2.995]
 [ 3.939  1.197  0.325 -0.744 -2.205]
 [ 1.348  0.055 -0.83  -0.836 -1.662]]
